In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR
import warnings
warnings.filterwarnings('ignore')

In [ ]:
custom_column_names = ['WSR0','WSR1']
df = pd.read_csv('eighthr.csv', parse_dates=[0], na_values=['?'],
                 index_col=0, names = (['WSR0', 'WSR1']+list(range(3, 74))))
df = df.dropna()
df.head()

In [ ]:
df['WSR0'] = df['WSR0'].astype(float)
df['WSR1'] = df['WSR1'].astype(float)

In [ ]:
fig,axes = plt.subplots(2,1,figsize=(14,8))
df['WSR1'].plot(ax=axes[0],title='WSR0')
df['WSR0'].plot(ax=axes[1],title='WSR1')
plt.show()

In [ ]:
result = adfuller(df['WSR0'])
print(result)
if result[1]<0.05:
    print("It is Stationary")
else:
    print("It is not stationary")

In [ ]:
result = adfuller(df['WSR1'])
print(result)
if result[1]<0.05:
    print("It is Stationary")
else:
    print("It is not stationary")

In [ ]:
print('WSR0 causes WSR1')
print('---------------------')
granger1=grangercausalitytests(df[['WSR0','WSR1']],2)
print('WSR1 causes WSR0')
print('---------------------')
granger1=grangercausalitytests(df[['WSR1','WSR0']],2)

In [ ]:
train = df[['WSR0','WSR1']]
model = VAR(train)
sortedmodel = model.select_order(maxlags=20)
sortedmodel.summary()

In [ ]:
model = VARMAX(df[['WSR0', 'WSR1']], order=(10,0),enforce_stationarity=True)
model_fit = model.fit()
model_fit.summary()

In [ ]:
n_forecast = 12
pred = model_fit.get_prediction()
preds = pred.predicted_mean

In [ ]:
preds.columns = ['WSR0 Predictions','WSR1 Predictions']
preds

In [ ]:
train = df[['WSR0','WSR1']]
testvspread = pd.concat([train,preds],axis=1)
testvspread

In [ ]:
testvspread[['WSR0','WSR0 Predictions']].plot()

In [ ]:
testvspread[['WSR1','WSR1 Predictions']].plot()

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(testvspread['WSR1'],testvspread['WSR1 Predictions'])

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(testvspread['WSR0'],testvspread['WSR0 Predictions'])